In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
train = pd.read_csv('train_clean.csv')

In [14]:
train['TripType'] = train['TripType'].astype('category')

In [4]:
train = pd.get_dummies( train, 'Weekday' )
X_train = train.iloc[:,2:len(train.columns)]

In [11]:
from sklearn.preprocessing import OneHotEncoder
one_hot_encoder = OneHotEncoder()
y_train = one_hot_encoder.fit_transform(train['TripType'][:, np.newaxis]).todense()

matrix([[1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 1.],
        [0., 0., 0., ..., 0., 0., 1.],
        [0., 0., 0., ..., 0., 0., 1.]])

In [5]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_train_scaled 

array([[0.14285714, 0.08695652, 0.25      , ..., 0.        , 0.        ,
        0.        ],
       [0.14285714, 0.08695652, 0.25      , ..., 0.        , 0.        ,
        0.        ],
       [0.14285714, 0.08695652, 0.25      , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.14285714, 0.08695652, 0.25      , ..., 0.        , 0.        ,
        0.        ],
       [0.14285714, 0.08695652, 0.25      , ..., 0.        , 0.        ,
        0.        ],
       [0.14285714, 0.08695652, 0.25      , ..., 0.        , 0.        ,
        0.        ]])

In [6]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn import metrics
from sklearn.metrics import accuracy_score

In [ ]:
import numpy as np
# coding: utf-8
from sklearn.feature_selection import SelectPercentile, f_regression
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import Ridge

## Generemos un data set de 100 observaciones y 1000 variables
## X y y serán independientes
rnd = np.random.RandomState(seed = 1234)
X = rnd.normal(size=(100, 10000))
y = rnd.normal(size=(100,))

## Recuerda que no deberíamos de aprender nada de este dataset

## Seleccionamos el percentil 5  de las variables
select = SelectPercentile(score_func=f_regression, percentile=5).fit(X,y)
X_selected = select.transform(X)

"R^2 Cross-Validation: {:.2f}".format(np.mean(cross_val_score(Ridge(), X_selected, y, cv=5)))

In [ ]:
seed = 1
classifier_names = ['LR',
                    'kNN',
                    'RBF SVM',
                    'Random Forest',
                    'Boosted Trees']
classifiers = [LogisticRegression(random_state=seed),
               KNeighborsClassifier(38),
               SVC(gamma=2, C=1, random_state=seed),
               RandomForestClassifier(max_depth=5, n_estimators=10, 
                                      max_features=1, random_state=seed),
               GradientBoostingClassifier(n_estimators=10, learning_rate=1.0,
                                          max_depth=5, random_state=seed)]
for model in classifiers:
    model.fit(X_train, train['TripType'])

In [ ]:
def test_roc(model, data, labels):
    if hasattr(model, "decision_function"):
        predictions = model.decision_function(data)
    else:
        predictions = model.predict_proba(data)[:,1]
    fpr, tpr, _ = sklearn.metrics.roc_curve(labels, predictions)
    return fpr, tpr

In [ ]:
plt.figure()
fpr_cluster, tpr_cluster = test_roc(lr_cluster, test_with_cluster, test_labels)
plt.plot(fpr_cluster, tpr_cluster, 'r-', label='LR with k-means')
print('LR with k-means AUC:{}'.format(sklearn.metrics.auc(fpr_cluster, tpr_cluster)))

for i, model in enumerate(classifiers):
    fpr, tpr = test_roc(model, test_data, test_labels)
    plt.plot(fpr, tpr, label=classifier_names[i])
    print(classifier_names[i]+' AUC:{}'.format(sklearn.metrics.auc(fpr, tpr)))

plt.plot([0, 1], [0, 1], 'k--')
plt.legend()
plt.xlabel('False Positive Rate', fontsize=14)
plt.ylabel('True Positive Rate', fontsize=14)

In [ ]:
## Matriz de confusión
from sklearn.metrics import confusion_matrix

mat = confusion_matrix(y_test, y_model)

plt.close()

sns.heatmap(mat, square = True, annot = True, cbar = False)
plt.xlabel('Valor predicho')
plt.ylabel('Valor real')